In [25]:
pip install google-generativeai

In [26]:
import os
os.environ["API_KEY"] =

for checking the API

In [27]:
import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

Elara, a girl with hair the color of dandelion fluff and eyes like the summer sky, lived in a world where stories whispered on the wind.  She longed for adventure, for the kind that filled dusty, forgotten books with tales of dragons and enchanted forests. One day, while rummaging in her grandmother's attic, she found it – a backpack, worn and leather-bound, with intricate silver stitching that seemed to shimmer with an inner light.

The moment Elara touched it, she felt a pull, a tingling sensation that ran up her arm. Inside, nestled amongst a faded map and a leather-bound journal, lay a single, shimmering feather. It pulsed with a warm light, and Elara knew, instinctively, it was magical. 

That night, the feather glowed brighter, the map unfurling itself, revealing a path that led to a shimmering, emerald forest. As Elara packed her backpack, it seemed to grow, its pockets multiplying, filled with everything she might need: a silver compass, a shimmering potion bottle, and a worn l

In [28]:
import os
import google.generativeai as genai
from google.cloud import language_v1
# 1. Define products and price range (at least 10)
products = {
    "Salmon (Wild Caught)": {"price": 20, "description": "A flavorful and healthy fish, perfect for grilling or baking."},
    "Tuna (Sushi Grade)": {"price": 30, "description": "Premium tuna, perfect for sashimi or sushi."},
    "Cod (Fresh)": {"price": 15, "description": "Mild-flavored fish, versatile for frying, baking, or steaming."},
    "Swordfish (Grilled)": {"price": 40, "description": "A hearty and flavorful fish, ideal for grilling or pan-searing."},
    "Halibut (Pan-Seared)": {"price": 50, "description": "A firm-fleshed fish with a delicate flavor, perfect for pan-searing or grilling."},
    "Grouper (Baked)": {"price": 60, "description": "A white, flaky fish with a mild flavor, delicious baked with herbs and spices."},
    "Snapper (Fried)": {"price": 35, "description": "A sweet and delicate fish, perfect for frying or grilling."},
    "Mahi-Mahi (Grilled)": {"price": 45, "description": "A firm, flavorful fish, ideal for grilling or broiling."},
    "Sea Bass (Steamed)": {"price": 55, "description": "A delicate and flavorful fish, perfect for steaming or poaching."},
    "Lobster (Steamed)": {"price": 100, "description": "A luxurious and flavorful crustacean, best enjoyed steamed or boiled."}
}

# 2. Establish negotiation rules
negotiation_rules = {
    "quantity": "Discount may be available for bulk purchases.",
    "delivery_time": "Rush orders may incur additional fees.",
    "payment_terms": "Flexible payment options may be available."
}

In [ ]:
def chatbot():
    genai.configure(api_key=os.environ["API_KEY"])
    model = genai.GenerativeModel("gemini-1.5-flash")
    client = language_v1.LanguageServiceClient()
    print("Welcome to the Fish Market!")
    print("We Serve", list(products.keys()))

    product_name = input("What kind of fish are you interested in? ")

    while True:
        budget = input("What is your budget? ")
        try:
            budget = int(budget)
            break
        except ValueError:
            print("Please enter a valid number for your budget.")

    # Find the product
    product = None
    for name, details in products.items():
        if name.lower() == product_name.lower():
            product = details
            break

    if product:
        print(f"You are interested in {product_name}.")
        print(f"Its price is {product['price']}.")
        print(f"Description: {product['description']}")

        # Start negotiation
        print("Do you want to negotiate the price? (yes/no)")
        negotiate = input().lower()

        if negotiate == "yes":
            print("Let's discuss your requirements. What factors can be adjusted?")

            # Quantity
            while True:
                quantity = input("How much fish do you need (in kg)? ")
                try:
                    quantity = float(quantity)
                    break
                except ValueError:
                    print("Please enter a valid number for quantity.")
            if quantity > 10:
                quantity_discount = 0.15  # 15% discount for bulk
            else:
                quantity_discount = 0

            # Delivery Time
            while True:
                delivery_time = input("When do you need the delivery? (today/1 week/after 1 week) ")
                if delivery_time.lower() in ["today", "1 week", "after 1 week"]:
                    break
                else:
                    print("Please choose from the options: today, 1 week, after 1 week.")
            if delivery_time.lower() == "today":
                delivery_discount = 0
            elif delivery_time.lower() == "1 week":
                delivery_discount = 0.15
            else:  # after 1 week
                delivery_discount = 0.18

            # Payment Terms
            while True:
                payment_terms = input("How would you like to pay? (prepaid/postpaid) ")
                if payment_terms.lower() in ["prepaid", "postpaid"]:
                    break
                else:
                    print("Please choose from the options: prepaid, postpaid.")
            if payment_terms.lower() == "prepaid":
                payment_discount = 0.05  # 5% discount for prepaid
            else:
                payment_discount = 0

            # Calculate total discount and final price
            total_discount = quantity_discount + delivery_discount + payment_discount
            if quantity > 10 and delivery_time.lower() != "today":  # Apply 20% if both quantity and delivery are true
                total_discount = 0.20

            final_price = product['price'] * (1 - total_discount)
            print(f"Your total discount is: {total_discount:.0%} ")
            print(f"Your final price is: {final_price:.2f}")
            # Check if user is okay with the price
            while True:
                okay_price = input("Are you okay with this price? (yes/no) ")
                if okay_price.lower() in ["yes", "no"]:
                    break
                else:
                    print("Please answer with yes or no.")

            if okay_price.lower() == "no":
                remark = input("Please tell me your concern: ")

                # Perform sentiment analysis
                document = language_v1.Document(content=remark, type_=language_v1.Document.Type.PLAIN_TEXT)
                sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
                sentiment_score = sentiment.score

                if sentiment_score < 0:  # Negative sentiment
                    # Suggest a price decrease based on sentiment score
                    suggestion = genai.generate_content(f"Suggest a price decrease based on the following remark: '{remark}'")
                    print(suggestion.text)
                else:
                    print("Sorry, we cannot fulfill this deal. We appreciate your understanding.")
        else:
            print("Thank you for your interest!")

    else:
        print(f"We do not have {product_name} in our inventory. Please choose from the following options:")
        for name in products.keys():
            print(name)
        chatbot()  # Restart the chatbot

chatbot()

Welcome to the Fish Market!
We Serve ['Salmon (Wild Caught)', 'Tuna (Sushi Grade)', 'Cod (Fresh)', 'Swordfish (Grilled)', 'Halibut (Pan-Seared)', 'Grouper (Baked)', 'Snapper (Fried)', 'Mahi-Mahi (Grilled)', 'Sea Bass (Steamed)', 'Lobster (Steamed)']
What kind of fish are you interested in? Cod (Fresh)
What is your budget? 11
You are interested in Cod (Fresh).
Its price is 15.
Description: Mild-flavored fish, versatile for frying, baking, or steaming.
Do you want to negotiate the price? (yes/no)
yes
Let's discuss your requirements. What factors can be adjusted?
How much fish do you need (in kg)? 12
When do you need the delivery? (today/1 week/after 1 week) today
How would you like to pay? (prepaid/postpaid) prepaid
Your total discount is: 20% 
Your final price is: 12.00
Are you okay with this price? (yes/no) no
Please tell me your concern: give me discount


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7af4446abdc0>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go